In [1]:
import yfinance as yf
import numpy as np

In [2]:
ohlcv_data={}

In [3]:
tickers=["AMZN","GOOG","MSFT"]

In [5]:
for ticker in tickers:
    temp=yf.download(ticker,period="1mo",interval="5m")
    temp.dropna(how="any",inplace=True)
    ohlcv_data[ticker] =temp

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [11]:
def ATR(DF,n=14):
    df=DF.copy()
    df["H-L"] = df["High"] - df["Low"]
    df["H-PC"]= abs(df["High"]-df["Adj Close"].shift(1))
    df["L-PC"]=abs(df["Low"]-df["Adj Close"].shift(1))
    df["TR"] = df[["H-L","H-PC","L-PC"]].max(axis=1, skipna = False)
    df["ATR"] = df["TR"].ewm(com=n,min_periods=n).mean()
    return df["ATR"]

In [23]:
def ADX(DF, n = 20):
    df = DF.copy()
    df["ATR"] = ATR(df,n)
    df["upmove"] = df["High"] - df["High"].shift(1)
    df["downmove"] = df["Low"].shift(1) - df["Low"]
    df["+dm"] = np.where((df["upmove"]>df["downmove"]) & (df["upmove"] > 0),df["upmove"],0)
    df["-dm"] = np.where((df["downmove"]>df["upmove"]) & (df["downmove"] > 0),df["downmove"],0)
    df["+di"] = 100 * (df["+dm"]/df["ATR"]).ewm(com = n,min_periods=n).mean()
    df["-di"] = 100 * (df["-dm"]/df["ATR"]).ewm(com = n,min_periods=n).mean()
    df["ADX"] = 100* abs((df["+di"] - df["-di"])/(df["+di"] + df["-di"])).ewm(com = n,min_periods=n).mean()
    return df["ADX"]

In [24]:
for ticker in ohlcv_data:
    ohlcv_data[ticker]["ADX"] = ADX(ohlcv_data[ticker],20)

In [25]:
ohlcv_data

{'AMZN':                                   Open         High          Low        Close  \
 Datetime                                                                        
 2022-01-03 10:05:00-05:00  3344.774902  3346.560059  3341.340088  3342.750000   
 2022-01-03 10:10:00-05:00  3343.645020  3354.050049  3342.260010  3354.050049   
 2022-01-03 10:15:00-05:00  3354.149902  3357.979980  3351.169922  3356.000000   
 2022-01-03 10:20:00-05:00  3355.709961  3362.975098  3350.419922  3362.139893   
 2022-01-03 10:25:00-05:00  3362.729980  3380.860107  3362.399902  3375.239990   
 ...                                ...          ...          ...          ...   
 2022-02-03 09:45:00-05:00  2851.000000  2855.675781  2846.010010  2847.074951   
 2022-02-03 09:50:00-05:00  2848.000000  2856.280029  2835.530029  2838.929932   
 2022-02-03 09:55:00-05:00  2837.445068  2843.415039  2818.462891  2818.462891   
 2022-02-03 10:00:00-05:00  2818.500000  2825.989990  2814.000000  2821.120117   
 2022-02